### Pandas ile Veri Kaynakların Bağlanma
* read_ ile başlayanlar okuyoruz. to_ ile başlayanlarla yazıyoruz.

In [1]:
# Yas kolonundan önce boşluk var.
# Veli öncesi bir satır boş geçilmiş.
# Tarih kolonu var.
# ondalık ayracı , olan bir değer var
# boş geçilmiş alanlar var
# boş geçilmiş alanlar için Yok ifadesi kullanılmış.

metin = '''
Ad|Soyad| Yas|Tarih|Kilo
Ahmet|Örnek|80|2022-03-24|70,2
Ümit|Veri|55|2022-03-23|

Veli|Çalışkan|30|Yok|Yok
'''

In [2]:
import io
print(io.StringIO(metin).read())


Ad|Soyad| Yas|Tarih|Kilo
Ahmet|Örnek|80|2022-03-24|70,2
Ümit|Veri|55|2022-03-23|

Veli|Çalışkan|30|Yok|Yok



### Pandas ile CSV okuma

In [3]:
import numpy as np
import pandas as pd

In [4]:
pd.read_csv(io.StringIO(metin),
           delimiter="|",
        # sep= "|"
           )

,Ad,Soyad,Yas,Tarih,Kilo
0,Ahmet,Örnek,80,2022-03-24,"70,2"
1,Ümit,Veri,55,2022-03-23,NaN
2,Veli,Çalışkan,30,Yok,Yok


In [5]:
df = pd.read_csv(io.StringIO(metin),
                 delimiter="|",
                 #usecols=["Ad", "Soyad"], #istediğimiz kolonlar
                 skipinitialspace=True, #kolon adlarındaki boşluklar görmezden gelinir.
                 #usecols=["Ad","Yas"]
                 #skiprows=1,
                 #skipfooter=1,
                 #skip_blank_lines=False,
                 na_values={"Yok"}, # Yok ifadesi NaN olacak
                 decimal=",", # floata çevirme
                 index_col=["Ad"], # index etiketi
                 parse_dates=['Tarih'],
                 dtype={
                         "Ad":str,
                         "Soyad":str,
                         "Yas":np.int8,
                         "Kilo":np.float16
                 }

           )
df
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, Ahmet to Veli
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Soyad   3 non-null      object        
 1   Yas     3 non-null      int8          
 2   Tarih   2 non-null      datetime64[ns]
 3   Kilo    1 non-null      float16       
dtypes: datetime64[ns](1), float16(1), int8(1), object(1)
memory usage: 81.0+ bytes


#### to_csv ile yazalım

In [6]:
import os

os.makedirs("databases", exist_ok=True) # klasör oluşturma

In [7]:
df.to_csv("databases/kisiler.csv")

#### converter
* belirtilen kolonda verilen fonksiyonu çalıştırır.
* veri temizliği için kullanılabilir. okurken temizlik yapabiliriz.

In [8]:
pd.read_csv("databases/kisiler.csv",
           converters={"Soyad":str.upper} # soyad kolonundaki değerler str.upper fonksiyonuna parametre olarak gelir
           )

,Ad,Soyad,Yas,Tarih,Kilo
0,Ahmet,ÖRNEK,80,2022-03-24,70.2
1,Ümit,VERI,55,2022-03-23,NaN
2,Veli,ÇALIŞKAN,30,NaN,NaN


In [9]:
#örn:
#soyadları düzeltelim. "Uçan":"Uçanlar", "Kaçan":"Kaçanoğulları"

temizlikKurali = {"Uçan":"Uçanlar", "Kaçan":"Kaçanoğulları"}

pd.read_csv("databases/kisiler.csv",
           converters= {"Soyad":lambda  v: temizlikKurali[v] if temizlikKurali.get(v) else v}
           )

,Ad,Soyad,Yas,Tarih,Kilo
0,Ahmet,Örnek,80,2022-03-24,70.2
1,Ümit,Veri,55,2022-03-23,NaN
2,Veli,Çalışkan,30,NaN,NaN


#### Excel'e bağlanmak

In [20]:
df_dict = pd.read_excel("databases/mykisiler.xlsx",
             sheet_name=None #tüm sayfalar okunur. sayfa adı verilebilir.
             )
df_dict

{'kisiler':                Id     Ad     Soyad  Yas      Tarih  Kilo
 0               1  Ahmet     Örnek   15 2022-03-24  70.2
 1               2   Ümit      Veri   22 2022-03-23   NaN
 2               3   Veli  Çalışkan   55        NaT   NaN,
 'Boş': Empty DataFrame
 Columns: []
 Index: []}

In [21]:
df_dict["kisiler"]

,Id,Ad,Soyad,Yas,Tarih,Kilo
0,1,Ahmet,Örnek,15,2022-03-24,70.2
1,2,Ümit,Veri,22,2022-03-23,NaN
2,3,Veli,Çalışkan,55,NaT,NaN


In [22]:
df_dict["Boş"]

""


In [13]:
#https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data

### RDBMS'e Bağlanmak

* `read_sql_query`: DBAPI destekliyor. `pyodbc` kullanabiliriz. `sqlachemy` ile çalışabilir. SQL sorgusu ister
* `read_sql`: DBAPI destekliyor. `pyodbc` kullanabiliriz. `sqlachemy` ile çalışabilir. Diğer iki fonksiyonu kapsar. SQL sorgusu veya tablo adı
* `read_sql_table`: `sqlalchemy` ile çalışır. sql tablosu ister

**Not:**
* `DBAPI` diğer tüm RDBMS'lere bağlanan kütüphanelerin alt yapısıdır.
* `sqlachemy` populer bir ORM (Object Relational Mapping) kütüphanesidir. Veritabanı nesneleri yazılım nesneleri olarak temsil edilir. 


### DBAPI ile

In [15]:
import pyodbc

pyodbc.drivers()

['SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)']

In [ ]:
con = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};Server=localhost;Database=AdventureWorks2012;Trusted_Connection=yes;")

In [ ]:
df = pd.read_sql_query(" SELECT Name, Color FROM Production.Product WHERE ListPrice > 0 ", con)

df.head()

### sqlalchemy

In [18]:
import sqlalchemy as sqla

In [ ]:
#engine oluşturmak gerekli
#!!!! ?Trusted_Connected=True ifadesini kaldırıyoruz.
con = sqla.create_engine("mssql+pyodbc://localhost/AdventureWorks2012?driver=ODBC Driver 17 for SQL Server")

In [ ]:
con.table_names(schema="Production")

In [ ]:
#1.
pd.read_sql_query("SELECT Name, Color, ListPrice FROM Production.Product WHERE ListPrice>0",con).head()

#2.
pd.read_sql("SELECT Name, Color, ListPrice FROM Production.Product WHERE ListPrice>0",con).head()

#3.
#pd.read_sql_table(schema="Production", table_name="Product",con=con).head()

### Veritabanına yazma

In [14]:
kisiler = pd.read_csv("databases/kisiler.csv")
kisiler.head(2)
kisiler.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Ad      3 non-null      object 
 1   Soyad   3 non-null      object 
 2   Yas     3 non-null      int64  
 3   Tarih   2 non-null      object 
 4   Kilo    1 non-null      float64
dtypes: float64(1), int64(1), object(3)
memory usage: 248.0+ bytes


In [ ]:
kisiler.to_sql(name="gtrKisiler", schema="dbo",con=coon)

### style ve option
* dataframe görünümünü değiştirmemizi sağlar.
* https://pandas.pydata.org/docs/user_guide/style.html

In [22]:
dir(pd.options.display)

['chop_threshold',
 'colheader_justify',
 'column_space',
 'date_dayfirst',
 'date_yearfirst',
 'encoding',
 'expand_frame_repr',
 'float_format',
 'html',
 'large_repr',
 'latex',
 'max_categories',
 'max_columns',
 'max_colwidth',
 'max_info_columns',
 'max_info_rows',
 'max_rows',
 'max_seq_items',
 'memory_usage',
 'min_rows',
 'multi_sparse',
 'notebook_repr_html',
 'pprint_nest_depth',
 'precision',
 'show_dimensions',
 'unicode',
 'width']

In [23]:
pd.options.display.float_format = "{:,.2f}".format

df

,Soyad,Yas,Tarih,Kilo
Ad,,,,
Ahmet,Örnek,80,2022-03-24,70.19
Ümit,Veri,55,2022-03-23,NaN
Veli,Çalışkan,30,NaT,NaN


In [24]:
pd.set_option("display.max_row", None)
df

,Soyad,Yas,Tarih,Kilo
Ad,,,,
Ahmet,Örnek,80,2022-03-24,70.19
Ümit,Veri,55,2022-03-23,NaN
Veli,Çalışkan,30,NaT,NaN


In [ ]:
#stype
#dir(df.style)

#
#df.style.format("")
#df.style.highlight_null(null_color="yellow",subset="Color")
#df.style.bar(axis=0, subset="ListPrice")

#df.style.highlight_max(subset="ListPrice", color="red", axis=0)